In [1]:
import dataclasses
#from training import TrainingPipeline
from typing import List, Dict

@dataclasses.dataclass
class opts:
    val_seed: int = 2
    canonical_test_seed: int = 1
    val_fraction: float = 0.2
    test_fraction: float = 0.2
    n_conv_layers: int = 2
    activation_layer: str = "ReLU"
    dropout_ratio: float = 0.5
    mplayer: str = "GCNConv"
    mplayer_args: List[str] = dataclasses.field(default_factory=list)
    mplayer_kwargs: Dict = dataclasses.field(default_factory=dict)
    latent_dim: int = 32
    layer_ratio: int = 10
    root: str = "./data/"
    dataset: str = "shalek"
    model_path: str = "./models/"
    lr: float = 1e-3
    es_patience: int = 10
    decoder: str = "InnerProductDecoder"
    val_mode: str = "max"
    val_metric: str = "average_precision_score"
    test_metrics: List[str] = dataclasses.field(default_factory=list)
    n_folds: int = 5
    epochs: int = 500
    cuda: bool = False
    negative_sampling: str = "pot_net"
    score_batched: bool = True
    wandb_tracking: bool = True
    wandb_project: str = "granpy-dev"
    wandb_save_model: bool = True
    wandb_group: str = None
    cache_model: bool = False

_opts = opts()

_opts.test_metrics += ["average_precision_score", "roc_auc_score"]

In [2]:
from src.datasets.datasets import McCallaDataset

dataset = McCallaDataset(root="./data/", opts = _opts, hash="abc")

print(dataset.train_data)

Data(x=[9411, 1211], edge_index=[2, 115494], num_nodes=9411, edge_label=[115494], edge_label_index=[2, 115494])


In [3]:
# single run

from src.experiment import Experiment
experiment = Experiment(_opts)
experiment.run()

CUDA is set to False, using cpu as fallback
CUDA is set to False, using cpu as fallback


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: marcostock (scialdonelab). Use `wandb login --relogin` to force relogin


Best average_precision_score: 0.8457574402455681:   2%|▏         | 12/500 [00:12<08:32,  1.05s/it]
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


average_precision_score,▁
roc_auc_score,▁
train_loss,████▇▇▆▅▅▃▂▁▃
val_average_precision_score,▁▅██▇▇▇▇▇▅▄▄▅
val_loss,██▇▇▇▆▅▅▄▃▃▂▁
average_precision_score,0.38433
roc_auc_score,0.75324
train_loss,1.4471
val_average_precision_score,0.84516
val_loss,1.4466


{'average_precision_score': 0.3843313111830709,
 'roc_auc_score': 0.7532439527419835}

In [4]:
# folded runs

from src.experiment import ExperimentArray
experimentArray = ExperimentArray(_opts)
experimentArray.run()

  0%|          | 0/5 [00:00<?, ?it/s]Processing...


NO CACHE FOUND - Processing data...


Done!


CUDA is set to False, using cpu as fallback
CUDA is set to False, using cpu as fallback


Best average_precision_score: 0.8460961859243717:   3%|▎         | 14/500 [00:16<09:44,  1.20s/it]
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


...

CUDA is set to False, using cpu as fallback
CUDA is set to False, using cpu as fallback


Best average_precision_score: 0.8484980009917636:   3%|▎         | 13/500 [00:12<07:55,  1.02it/s]
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


average_precision_score,▁
roc_auc_score,▁
train_loss,██▇▇▇▆▆▅▅▄▃▂▁▁
val_average_precision_score,▅▇██▇▆▅▄▃▂▁▁▂▂
val_loss,██▇▇▇▆▆▅▅▄▃▃▂▁
average_precision_score,0.38474
roc_auc_score,0.75361
train_loss,1.44656
val_average_precision_score,0.84601
val_loss,1.44606


100%|██████████| 5/5 [02:15<00:00, 27.09s/it]


[{'average_precision_score': 0.3842363860774682,
  'roc_auc_score': 0.7533069456898615},
 {'average_precision_score': 0.3860045051348083,
  'roc_auc_score': 0.7534996485437784},
 {'average_precision_score': 0.3845679411458947,
  'roc_auc_score': 0.7534722618221762},
 {'average_precision_score': 0.38629081942510063,
  'roc_auc_score': 0.7534810406922077},
 {'average_precision_score': 0.38473665741299845,
  'roc_auc_score': 0.7536084802105218}]

In [3]:
#hyperparameter sweep
import wandb 
from src.experiment import Experiment

def singleRun():
    experiment = Experiment(_opts)
    experiment.run()

sweep_configuration = {
    "method": "random",
    "metric": {"goal": "minimize", "name": "val_loss"},
    "parameters": {
        "dropout_ratio": {"max": 0.6, "min": 0.1},
        "decoder": {"values": ["InnerProductDecoder", "CosineDecoder", "PNormDecoder"]},
        "n_conv_layers": {"values": [1,2,3]},
    },
}
sweep_id = wandb.sweep(sweep=sweep_configuration, project=opts.wandb_project, entity="scialdonelab")
wandb.agent(sweep_id, function=singleRun, count=10)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: 8ge4oulx
Sweep URL: https://wandb.ai/scialdonelab/granpy-dev/sweeps/8ge4oulx


wandb: Agent Starting Run: jbxirydh with config:
wandb: 	decoder: InnerProductDecoder
wandb: 	dropout_ratio: 0.4752309561780738
wandb: 	n_conv_layers: 2
Processing...


NO CACHE FOUND - Processing data...


Done!


...

CUDA is set to False, using cpu as fallback
CUDA is set to False, using cpu as fallback


wandb: Agent Starting Run: xli9y40r with config:
wandb: 	decoder: PNormDecoder
wandb: 	dropout_ratio: 0.1044556795397232
wandb: 	n_conv_layers: 2


CUDA is set to False, using cpu as fallback
CUDA is set to False, using cpu as fallback


wandb: Agent Starting Run: j4oyaidj with config:
wandb: 	decoder: CosineDecoder
wandb: 	dropout_ratio: 0.10472663873924767
wandb: 	n_conv_layers: 1


CUDA is set to False, using cpu as fallback
CUDA is set to False, using cpu as fallback


In [ ]:
import wandb

api = wandb.Api()

# run is specified by <entity>/<project>/<run_id>
run = api.run("scialdonelab/granpy-dev/<run_id>")

# save the metrics for the run to a csv file
metrics_dataframe = run.history()